In [ ]:
from sklearn.cluster import KMeans

import numpy as np 
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# Convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# Map rendering library
import folium 

import requests 
import json 

# Library to parse HTML and XML documents
from bs4 import BeautifulSoup 

# Transform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 